# Imports

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
!git clone https://github.com/Blu-FiBEr/Scoutify-Helper

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import plotly.express as px
import plotly
import math

# Reading Data

In [ ]:
df_17 = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/FIFA17_official_data.csv')
df_18 = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/FIFA18_official_data.csv')
df_19 = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/FIFA19_official_data.csv')
df_20 = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/FIFA20_official_data.csv')
df_21 = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/FIFA21_official_data.csv')
df_22 = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/FIFA22_official_data.csv')
datalist = [df_17,df_18,df_19,df_20,df_21,df_22]

In [ ]:
df_22.columns

# Cleaning Data

In [ ]:
curr_set = df_17
for next_set in datalist[1:]:
    print(set(next_set) - set(curr_set))
    curr_set = next_set

for dataset in datalist:
    if 'Release Clause' in list(dataset.columns):
        dataset.drop("Release Clause", axis = 1, inplace = True)
    if 'DefensiveAwareness' in list(dataset.columns):
        dataset.drop("DefensiveAwareness", axis = 1, inplace = True)
print("--------------------")
curr_set = df_17
for next_set in datalist[1:]:
    print(set(next_set) - set(curr_set))
    curr_set = next_set

In [ ]:
#Now we can join the datasets\
F_all_df = pd.concat(datalist, ignore_index=True)
F_all_df.columns

In [ ]:
Unimportant_features  = ["Photo", "Flag", "Club Logo", "Wage", "Special", "International Reputation", "Work Rate", "Body Type", "Real Face", "Jersey Number", "Contract Valid Until", "Best Overall Rating","Joined","Loaned From"]
F_all_df.drop(Unimportant_features, axis = 1, inplace = True)
F_all_df["Ovr_pot"] = (F_all_df["Overall"] + F_all_df["Potential"])/2

In [ ]:
# One-hot encode Preferred_foot
F_all_df = pd.get_dummies(F_all_df, columns = ["Preferred Foot"])
F_all_df

In [ ]:
detailed_features = ['Age','Overall', 'Potential', 'Weak Foot', 'Skill Moves', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', "Ovr_pot", 'Preferred Foot_Left', 'Preferred Foot_Right']

In [ ]:
F_all_df[detailed_features] = F_all_df[detailed_features].fillna(0)
F_all_df

In [ ]:
# Clean height column
feet_inches_re = re.compile(r"(\d)'(\d+)")

# Define a function to convert feet'inches to cm
def feet_inches_to_cm(s):
    match = feet_inches_re.match(s)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        return round((feet*12 + inches) * 2.54, 2)
    else:
        return float(s.replace('cm', ''))
F_all_df['Height'] = pd.to_numeric(F_all_df['Height'].apply(feet_inches_to_cm))

In [ ]:
F_all_df['Height'].value_counts()

In [ ]:
# Clean Weight Column
def convert_weight(weight_str):
    if 'kg' in weight_str:
        # If weight is in kg, remove the 'kg' suffix and return as float
        return float(weight_str.replace('kg', ''))
    elif 'lbs' in weight_str:
        # If weight is in lbs, convert to kg and return as float
        return float(weight_str.replace('lbs', '')) * 0.453592
    else:
        print("oops")
        # Return NaN if weight format is not recognized
        return pd.np.nan

# Apply the function to the Weight column and create a new column 'Weight_kg'
F_all_df['Weight'] = F_all_df['Weight'].apply(convert_weight)

In [ ]:
F_all_df['Weight'].value_counts()

In [ ]:
# Define a function to convert cost to millions
def convert_cost(cost_str):
    if 'M' in cost_str:
        # If cost is in millions, remove the '$' and 'M' suffix and return as float
        return float(cost_str.replace('€', '').replace('M', ''))
    elif 'K' in cost_str:
        # If cost is in thousands, remove the '$' and 'K' suffix, divide by 1000, and return as float
        return float(cost_str.replace('€', '').replace('K', '')) / 1000
    # else:
    #     # Return NaN if cost format is not recognized
    #     return pd.np.nan

# Apply the function to the Cost column and create a new column 'Cost_millions'
F_all_df['Value'] = F_all_df['Value'].apply(convert_cost)


# Feature Creation - BMI

In [ ]:
F_all_df['Bmi'] = F_all_df['Weight'] / (F_all_df['Height']/100)**2

In [ ]:
F_all_df.columns

# EDA

In [ ]:
# fig = go.Figure()

# fig = go.Figure(data=go.Scatter(
#     x = df_20['Overall'],
#     y = df_20['Value'],
#     mode='markers',
#     marker=dict(
#         size=16,
#         color=df_20['Age'], #set color equal to a variable
#         colorscale='Plasma', # one of plotly colorscales
#         showscale=True
#     ),
#     text= df_20['Name'],
# ))

# fig.update_layout(title='Styled Scatter Plot (colored by Age) year 2020 - Overall Rating vs Value in Euros',
#                   xaxis_title='Overall Rating',
#                   yaxis_title='Value in Euros',
#                   paper_bgcolor='rgba(0,0,0,0)',
#                   plot_bgcolor='rgba(0,0,0,0)',
#                   font=dict(family='Cambria, monospace', size=12, color='#000000'))
# fig.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = sklearn.preprocessing.StandardScaler()
F_norm_df = F_all_df.copy(deep = True)
F_norm_df[detailed_features] = scaler.fit_transform(F_norm_df[detailed_features].to_numpy())
F_norm_df.columns

In [ ]:
df_22 = F_all_df.tail(df_22.shape[0])

In [ ]:
df_22

In [ ]:
# fig = go.Figure()
# sample = F_norm_df.sort_values(by='Ovr_pot')

# fig.add_trace(go.Box(
#     x = sample['Ovr_pot'],
#     y = sample['Bmi'],
#     name="Suspected Outliers",
#     boxpoints='suspectedoutliers', # only suspected outliers
#     marker=dict(
#         size=12,
#         color='rgb(251, 158, 58)',
#         outliercolor='rgba(216, 87, 107, 0.6)',
#         line=dict(
#             outliercolor='rgba(216, 87, 107, 0.6)',
#             outlierwidth=2)),
#     line_color='rgb(73, 3, 159)',
#     text= sample['Name']
# ))

# fig.update_layout(title='Styled Box Plot (with Suspected Outliers) - Overall Rating vs BMI',
#                   xaxis_title='Overall Rating',
#                   yaxis_title='BMI',
#                   paper_bgcolor='rgba(0,0,0,0)',
#                   plot_bgcolor='rgba(0,0,0,0)',
#                   font=dict(family='Cambria, monospace', size=12, color='#000000'),
#                   xaxis_rangeslider_visible=True)
# fig.show()

In [ ]:
F_norm_df.columns

In [ ]:
F_norm_df["Best Position"].value_counts()

# Strikers

In [ ]:
striker_df = F_norm_df[(F_norm_df["Best Position"] == "ST") | (F_norm_df["Best Position"] == "CF") | (F_norm_df["Best Position"] == "LF")]
striker_df  =  striker_df.reset_index(drop = True)
striker_df

In [ ]:
ST_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X = striker_df[ST_detailed_features].to_numpy()
y = striker_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
np.sort(fs.scores_)

In [ ]:
ST_essential_ft = list(fs.get_feature_names_out(ST_detailed_features))
ST_essential_ft

# Wingers

In [ ]:
wingers_df = F_norm_df[(F_norm_df["Best Position"] == "LW") | (F_norm_df["Best Position"] == "RW")]
wingers_df  =  wingers_df.reset_index(drop = True)
wingers_df

In [ ]:
Wing_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
X = wingers_df[Wing_detailed_features].to_numpy()
y = wingers_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
np.sort(fs.scores_)

In [ ]:
Wing_essential_ft = list(fs.get_feature_names_out(Wing_detailed_features))
Wing_essential_ft

# Midfield

In [ ]:
mids_df = F_norm_df[(F_norm_df["Best Position"] == "RM") | (F_norm_df["Best Position"] == "CM") | (F_norm_df["Best Position"] == "CDM") | (F_norm_df["Best Position"] == "LM")]
mids_df  =  mids_df.reset_index(drop = True)
mids_df

In [ ]:
Mid_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
X = mids_df[Mid_detailed_features].to_numpy()
y = mids_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
Mid_essential_ft = list(fs.get_feature_names_out(Mid_detailed_features))
Mid_essential_ft

# CAM

In [ ]:
cam_df = F_norm_df[(F_norm_df["Best Position"] == "CAM")]
cam_df  =  cam_df.reset_index(drop = True)
cam_df

In [ ]:
Cam_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
X = cam_df[Cam_detailed_features].to_numpy()
y = cam_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
Cam_essential_ft = list(fs.get_feature_names_out(Cam_detailed_features))
Cam_essential_ft

# Defenders

In [ ]:
centrebacks_df = F_norm_df[(F_norm_df["Best Position"] == "CB")]
centrebacks_df  = centrebacks_df.reset_index(drop = True)
centrebacks_df

In [ ]:
Def_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
X = centrebacks_df[Def_detailed_features].to_numpy()
y = centrebacks_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
np.sort(fs.scores_)

In [ ]:
Def_essential_ft = list(fs.get_feature_names_out(Def_detailed_features))
Def_essential_ft

# Fullbacks

In [ ]:
fullbacks_df = F_norm_df[(F_norm_df["Best Position"] == "LB") | (F_norm_df["Best Position"] == "RB") | (F_norm_df["Best Position"] == "LWB") | (F_norm_df["Best Position"] == "RWB")]
fullbacks_df  = fullbacks_df.reset_index(drop = True)
fullbacks_df

In [ ]:
Def_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
X = fullbacks_df[Def_detailed_features].to_numpy()
y = fullbacks_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
Fullback_essential_ft = list(fs.get_feature_names_out(Def_detailed_features))
Fullback_essential_ft

# Goalkeeper

In [ ]:
gk_df = F_norm_df[(F_norm_df["Best Position"] == "GK")]
gk_df  = gk_df.reset_index(drop = True)
gk_df

In [ ]:
Gk_detailed_features = ['Height', 'Weight','Bmi', 'ShortPassing', 'Volleys', 'Curve','Agility', 'Reactions', 'Balance',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
        'Positioning', 'Vision', 'Composure','GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']

In [ ]:
gk_df[gk_df["ID"]==167495]


In [ ]:
X = gk_df[Gk_detailed_features].to_numpy()
y = gk_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [ ]:
Gk_essential_ft = list(fs.get_feature_names_out(Gk_detailed_features))
Gk_essential_ft

# Finalizing all dataframes

In [ ]:
# gk_df=gk_df[Gk_essential_ft]
# fullbacks_df = fullbacks_df[Fullback_essential_ft]
# centrebacks_df = centrebacks_df[Def_essential_ft]
# cam_df = cam_df[Cam_essential_ft]
# mids_df = mids_df[Mid_essential_ft]
# wingers_df = wingers_df[Wing_essential_ft]
# striker_df= striker_df[ST_essential_ft]

In [ ]:
# F_all_df["Club"].unique().shape

In [ ]:
# F_all_df[(F_all_df["Club"] == "Manchester United")].columns

In [ ]:
all_clubs = list(F_all_df["Club"].unique())

In [ ]:
all_clubs

# Previous players in a club

In [ ]:
# strikers_in_club={}

# for club in all_clubs:
#     all_features = striker_df[striker_df["Club"] == club]
#     essential_features = all_features[ST_essential_ft]
#     weights_vector=[]
#     for i in essential_features:
#         if essential_features[i].std() !=0:
#             weights_vector.append(1/essential_features[i].std())
#         else:
#             weights_vector.append(0)
#     # print(weights_vector)
#     strikers_in_club[club] = weights_vector
    





# Get weights of a particular position for all clubs

Weights are how dense the essential feature is. This is reciprocal of standard deviation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def get_pos_weights(pos_df,pos_essential_ft):
    pos_players_in_club={}

    for club in all_clubs:
        all_features = pos_df[pos_df["Club"] == club]
        all_features = all_features.sort_values('Ovr_pot',ascending = False).head(20)
        essential_features = all_features[pos_essential_ft]
        weights_vector=[]
        sum = 0
        for i in essential_features:
            if essential_features[i].std() != 0:
                weights_vector.append(1/essential_features[i].std())
                sum += 1/essential_features[i].std()
            else:
                weights_vector.append(0)
                sum += 0
        # print(pos_essential_ft,club,weights_vector)
        if(sum != 0):
            for x in range(len(weights_vector)):
                weights_vector[x] = weights_vector[x] / sum
        pos_players_in_club[club] = weights_vector
        #print(sum)
    np_array = np.empty((0,len(pos_essential_ft)))
    for club in all_clubs:
        n_row = np.array(pos_players_in_club[club])
        n_row.shape = (1,len(pos_essential_ft))
        np_array = np.append(np_array, n_row, axis = 0)
    scaler = StandardScaler()
    np_array = scaler.fit_transform(np_array)
    count = 0
    for club in all_clubs:
        scaler = MinMaxScaler()
        n_ar = np_array[count]
        n_ar.shape = (len(pos_essential_ft),1)
        n_ar = scaler.fit_transform(n_ar)
        n_ar.shape = (len(pos_essential_ft),)
        pos_players_in_club[club] = (n_ar/np.sum(n_ar)).tolist()
        count += 1

    return pos_players_in_club



# Extracting Common features across positions

In [ ]:
pos_essential_features = [ST_essential_ft,Wing_essential_ft,Mid_essential_ft,Cam_essential_ft,Def_essential_ft,Fullback_essential_ft]
# common_features =   []
# d={}
# for i in pos_essential_features:
#     for j in i:
#         if j not in d:
#             d[j]=1
#         else:
#             d[j]+=1

# for i in d:
#     if d[i]==len(pos_essential_features):
#         common_features.append(i)

#Can be optimized 

#basically removing all common features from the essential features of a position
# for i in common_features:
#     ST_essential_ft.remove(i)
#     Wing_essential_ft.remove(i)
#     Mid_essential_ft.remove(i)
#     Cam_essential_ft.remove(i)
#     Def_essential_ft.remove(i)
#     Fullback_essential_ft.remove(i)
    # Gk_essential_ft.remove(i)





In [ ]:
# common_features

In [ ]:
#common_features.extend(["Age", "Height", "Weight", "Weak Foot"])
common_features = (["Age"])

More features to be appended to common features.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = sklearn.preprocessing.MinMaxScaler()
F_std_df = F_all_df.copy(deep = True)
F_std_df[detailed_features] = scaler.fit_transform(F_std_df[detailed_features].to_numpy())
F_std_df.columns

# Weights per Position

In [ ]:
striker_df = F_std_df[(F_std_df["Best Position"] == "ST") | (F_std_df["Best Position"] == "CF") | (F_std_df["Best Position"] == "LF")]
striker_df  =  striker_df.reset_index(drop = True)
striker_df

In [ ]:
df_22_std = F_std_df.tail(df_22.shape[0])

In [ ]:
wingers_df = F_std_df[(F_std_df["Best Position"] == "LW") | (F_std_df["Best Position"] == "RW")]
wingers_df  =  wingers_df.reset_index(drop = True)
wingers_df

In [ ]:
mids_df = F_std_df[(F_std_df["Best Position"] == "RM") | (F_std_df["Best Position"] == "CM") | (F_std_df["Best Position"] == "CDM") | (F_norm_df["Best Position"] == "LM")]
mids_df  =  mids_df.reset_index(drop = True)
mids_df

In [ ]:
cam_df = F_std_df[(F_std_df["Best Position"] == "CAM")]
cam_df  =  cam_df.reset_index(drop = True)
cam_df

In [ ]:
centrebacks_df = F_std_df[(F_std_df["Best Position"] == "CB")]
centrebacks_df  = centrebacks_df.reset_index(drop = True)
centrebacks_df

In [ ]:
fullbacks_df = F_std_df[(F_std_df["Best Position"] == "LB") | (F_std_df["Best Position"] == "RB") | (F_std_df["Best Position"] == "LWB") | (F_std_df["Best Position"] == "RWB")]
fullbacks_df  = fullbacks_df.reset_index(drop = True)
fullbacks_df

In [ ]:
gk_df = F_std_df[(F_std_df["Best Position"] == "GK")]
gk_df  = gk_df.reset_index(drop = True)
gk_df

In [ ]:
import pickle

In [ ]:
# striker_weights_per_club = get_pos_weights(striker_df,ST_essential_ft)
# with open('./rec_actual_data/striker_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(striker_weights_per_club, f)

In [ ]:
# wing_weights_per_club = get_pos_weights(wingers_df,Wing_essential_ft)
# with open('./rec_actual_data/wing_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(wing_weights_per_club, f)

In [ ]:
# mid_weights_per_club = get_pos_weights(mids_df,Mid_essential_ft)
# with open('./rec_actual_data/mid_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(mid_weights_per_club, f)

In [ ]:
# cam_weights_per_club = get_pos_weights(cam_df,Cam_essential_ft)
# with open('./rec_actual_data/cam_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(cam_weights_per_club, f)

In [ ]:
# def_weights_per_club = get_pos_weights(centrebacks_df,Def_essential_ft)
# with open('./rec_actual_data/def_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(def_weights_per_club, f)

In [ ]:
# fullback_weights_per_club = get_pos_weights(fullbacks_df,Fullback_essential_ft)
# with open('./rec_actual_data/fullback_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(fullback_weights_per_club, f)

In [ ]:
# gk_weights_per_club = get_pos_weights(gk_df,Gk_essential_ft)
# with open('./rec_actual_data/gk_weights_per_club.pkl', 'wb') as f:
#     pickle.dump(gk_weights_per_club, f)

In [ ]:
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/striker_weights_per_club.pkl', 'rb') as f:
    striker_weights_per_club = pickle.load(f)
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/wing_weights_per_club.pkl', 'rb') as f:
    wing_weights_per_club = pickle.load(f)
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/mid_weights_per_club.pkl', 'rb') as f:
    mid_weights_per_club = pickle.load(f)
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/cam_weights_per_club.pkl', 'rb') as f:
    cam_weights_per_club = pickle.load(f)
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/def_weights_per_club.pkl', 'rb') as f:
    def_weights_per_club = pickle.load(f)
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/fullback_weights_per_club.pkl', 'rb') as f:
    fullback_weights_per_club = pickle.load(f)
with open('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/rec_actual_data/gk_weights_per_club.pkl', 'rb') as f:
    gk_weights_per_club = pickle.load(f)

In [ ]:
def_weights_per_club["Manchester City"]

# Common features weights

In [ ]:
# common_club_philosophy = get_pos_weights(F_std_df,common_features)

In [ ]:
# common_club_philosophy

# Current players in a club weighted rating


Get all current players of a club -  use df_22 as it is the most current season.
Then according to the player's overall rating give them weights and come up with a weighted average for the club.


In [ ]:
def get_cur_pos_philosophy(cur_season_df, club , pos_list,pos_essential_ft):# pos list will be ["LW","RW"] for WING
    club_df = cur_season_df[cur_season_df["Club"]==club]
    pos_df=club_df[club_df['Best Position'].isin(pos_list)]
    essential_df = pos_df[pos_essential_ft]
    #print(essential_df)
    cur_players = np.array(essential_df)
    denom=0
    num = np.zeros(shape = (cur_players.shape[1]))
    
    for i in range(cur_players.shape[0]):
        # print(pos_df.iloc[i])
        overall_rating = pos_df.iloc[i]["Ovr_pot"]
        #np.apply_along_axis(my_func, 0, overall_rating)
        
        # print("Overall rating: ", overall_rating)
        
        if(overall_rating >= 85):
            w = 20
        elif (overall_rating >= 80):
            w = 16
        elif(overall_rating >= 75):
            w = 8
        elif(overall_rating >= 70):
            w = 2
        else:
            w = 1
        denom+=w
        num+=w * cur_players[i]
        # print(num)
    
        
        

    return num/denom


In [ ]:
get_cur_pos_philosophy(F_std_df,"Manchester City", ["CB"],Def_essential_ft)

# Position similarity of a new player

similarity is basically the weighted average of the difference in new_player's attributes and current club players' attributes.

In [ ]:
from numpy import inf
from sklearn.preprocessing import normalize
def club_pos_sim(new_player,pos_essential_features,pos_weights_per_club, means, club_cur = None):
    new_player = np.array(new_player[pos_essential_features])
    new_player.shape = [new_player.shape[0],1]
    new_player = normalize(new_player, axis = 0)
    new_player.shape = [new_player.shape[0],]
    
    #print(new_player)
    
    # print(new_player.shape)
    # print(new_player)
    
    
    #club_cur  = get_cur_pos_philosophy(F_std_df,club,pos_list,pos_essential_features)
    club_cur.shape = [club_cur.shape[0],1]
    club_cur = normalize(club_cur, axis = 0)
    club_cur.shape = [club_cur.shape[0],]
    # print(club_cur)
    weighted_dot = np.sum(pos_weights_per_club * (new_player-means) * (club_cur-means))
    # diffs = np.reciprocal(np.square(np.array(new_player - club_cur)))
    # diffs[diffs == inf] = 10
    # #print(diffs)
    # pos_sim = diffs.dot(pos_weights_per_club[club])
    mag_1 = np.sqrt(np.dot(pos_weights_per_club, np.square(new_player - means)))
    mag_2 = np.sqrt(np.dot(pos_weights_per_club, np.square(club_cur - means)))
    pos_sim = weighted_dot/(mag_1 * mag_2)

    return pos_sim



In [ ]:
# new_player = df_22_std[df_22_std["ID"]==234576]#suarez
# # pos_np = pos_df.to_numpy()
# # pos_np = normalize(pos_np)
# # means = np.mean(pos_np, axis = 0)
# pos_sim = club_pos_sim(new_player,"Manchester City", ["CB"],Def_essential_ft,def_weights_per_club)

In [ ]:
# pos_sim

In [ ]:
def club_pos_mag(new_player,pos_essential_features,pos_weights_per_club):

    new_player = np.array(new_player[pos_essential_features])

    #new_player.shape = [new_player.shape[1],1]
    mag_1 = np.sqrt(np.dot(pos_weights_per_club, np.square(new_player)))

    return mag_1


In [ ]:
# new_player = df_22_std[df_22_std["ID"]==220806]#suarez
# pos_mag = club_pos_mag(new_player,"Manchester United",Def_essential_ft,def_weights_per_club)

In [ ]:
# pos_mag

In [ ]:
# ar_1 = np.array([1,2,3])
# ar_1.shape = [1,3]
# ar_2 = np.array([1,2,3])
# ar_1.shape = [1,3]
# ar_3 = np.array([1,2,3])
# ar_1.shape = [1,3]
# ar_1 * ar_2 * ar_3
# (ar_1).shape

In [ ]:
# n_p = np.array(new_player[Def_essential_ft])
# n_p.shape = [13,]
# np.sqrt(np.sum(def_weights_per_club["FC Barcelona"] * np.square(n_p)))


In [ ]:
# def get_cur_philosophy(cur_season_df,  club ,common_features):# pos list will be ["LW","RW"] for WING
#     club_df = cur_season_df[cur_season_df["Club"]==club]
#     #pos_df=club_df[club_df['Best Position'].isin(pos_list)]
#     essential_df = club_df[common_features]

#     cur_players = np.array(essential_df)
#     denom=0
#     num = np.zeros(shape = (cur_players.shape[1]))
#     for i in range(cur_players.shape[0]):
#         # print(pos_df.iloc[i])
#         overall_rating = club_df.iloc[i]["Ovr_pot"]
#         #np.apply_along_axis(my_func, 0, overall_rating)
        
#         # print("Overall rating: ", overall_rating)
        
#         if(overall_rating >= 85):
#             w = 20
#         elif (overall_rating >= 80):
#             w = 16
#         elif(overall_rating >= 75):
#             w = 8
#         elif(overall_rating >= 70):
#             w = 2
#         else:
#             w = 1
#         denom+=w
#         num+=w * cur_players[i]
#         # print(num)
    
        
        

#     return num/denom



In [ ]:
# get_cur_philosophy(F_std_df, "Manchester City", common_features)

## Similarity with common features/ common club philosophy

In [ ]:
# from numpy import inf
# #new_player = df_22[df_22["ID"]==176580]#suarez
# def club_common_sim(new_player,club,common_features, common_club_philosophy):
#     new_player = df_22[df_22["ID"] == new_player["ID"]]
#     new_player = np.array(new_player[common_features])
#     #new_player.shape = [new_player.shape[1],]
#     club_cur  = get_cur_philosophy(F_all_df,club,common_features)
#     #print(np.reciprocal((new_player - club_cur)))
#     diffs = np.reciprocal(np.absolute(np.array(new_player - club_cur)))
#     diffs[diffs == inf] = 1
#     #print(diffs)
#     club_sim = diffs
#     #club_sim/=sum(common_club_philosophy[club])
#     print(club_sim)
#     return (club_sim)


In [ ]:
# common_club_philosophy["Manchester City"]

In [ ]:
# new_player = df_22[df_22["ID"]==197061]

# club_sim = club_common_sim(new_player,"Manchester City",common_features, common_club_philosophy)
# club_sim

In [ ]:
# df_22[df_22["ID"]==203574]

In [ ]:
# get_cur_philosophy(F_all_df,"Manchester City",common_features)

# Recommendation based on similarity only

In [ ]:
def get_player_recs(df,club,pos_list, pos_essential_features,pos_weights_per_club, age_min, age_max, val,lambda1 = 0.73,lambda2 = 0.37):
    # print(club, age_min,age_max, val)
    pos_df=df[df['Best Position'].isin(pos_list)]
    essential_df = pos_df[pos_essential_features]
    # common_club_df = pos_df[common_club_features]
    d={}
    player_list = []
    min_sim = inf
    max_sim = -inf
    min_mag = inf
    max_mag = -inf
    #sum_sim = 0
    #sum_mag = 0
    # min_club = inf
    # max_club = -inf
    pos_np = essential_df.to_numpy()
    pos_np = normalize(pos_np)
    means = np.mean(pos_np, axis = 0)
    club_cur  = get_cur_pos_philosophy(F_std_df,club,pos_list,pos_essential_features)
    for i in range(pos_df.shape[0]):
        new_player = pos_df.iloc[i]
        pos_sim = club_pos_sim(new_player,pos_essential_features,pos_weights_per_club[club], means, club_cur = club_cur)
        if(min_sim > pos_sim): min_sim = pos_sim
        if(max_sim < pos_sim): max_sim = pos_sim
        #sum_sim += pos_sim
        pos_mag = club_pos_mag(new_player,pos_essential_features,pos_weights_per_club[club])
        if(min_mag > pos_mag): min_mag = pos_mag
        if(max_mag < pos_mag): max_mag = pos_mag
        #sum_mag += pos_mag
        #print(pos_sim, pos_mag)
        #club_sim = club_common_sim(new_player,club,common_club_features, common_club_philosophy)
        #print(club_sim)
        # if(min_club > club_sim): min_club = club_sim
        # if(max_club < club_sim): max_club = club_sim
        #sim = lambda1*pos_sim + lambda2*pos_mag
        d[str(new_player["ID"])] = [pos_sim, pos_mag]
        player_list.append(new_player["ID"])
    # mean_sim = sum_sim/pos_df.shape[0]
    # mean_mag = sum_mag/pos_df.shape[0]
    # dev_sum_sim = 0
    # dev_sum_mag = 0
    d_final = {}
    # for id in player_list:
    #     dev_sum_sim += (d[str(id)][0]- mean_sim)**2
    #     dev_sum_mag += (d[str(id)][1]- mean_mag)**2
    # std_sim = math.sqrt(dev_sum_sim/(pos_df.shape[0]-1))
    # std_mag = math.sqrt(dev_sum_mag/(pos_df.shape[0]-1))
    # min_sim = mean_sim - std_sim
    # min_mag = mean_mag - std_mag
    #print(min_sim, min_mag)
    for id in player_list:
        d[str(id)] = [(d[str(id)][0] - min_sim)/(max_sim - min_sim), (d[str(id)][1] - min_mag)/(max_mag - min_mag)]
        d_final[str(id)] = lambda1*d[str(id)][0] + lambda2*d[str(id)][1]
        #print(id, d[str(id)][0], d[str(id)][1])
    player_list.sort(key = lambda x: d_final[str(x)],reverse = True)
    new_list = [[df[df["ID"] == j]["Name"].values[0],j, d[str(j)]] for j in player_list[:] if ((float(df_22[df_22["ID"] == j]["Age"]) >= age_min) and (float(df_22[df_22["ID"] == j]["Age"]) <= age_max) and (float(df[df["ID"] == j]["Value"]) <= val) and ((df[df["ID"] == j]["Club"].values[0] != club)))]
    return new_list

In [ ]:
plslst = get_player_recs(df_22_std,"Manchester United",["CB"],Def_essential_ft,def_weights_per_club, age_min = 23, age_max = 27, val = 100)[0:10]

In [ ]:
plslst

In [ ]:
get_player_recs(df_22_std,"Manchester City",["CB"],Def_essential_ft,def_weights_per_club, age_min = 23, age_max = 27, val = 100)[0:10]

get_player_recs(df_22_std, "<CLUB NAME>", Defenders=>["CB"], Midfielders=>["LM","CM","RM", "CDM"], Strikers=>["ST","CF","LF"], Wingers=>[RW,LW], CAM=>["CAM"], Fullbacks=>[LB,RB,LWB,RWB], Goalkeeper=>["GK"], 



# UI

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
def cleanText(textlst):
  string =""
  i=1
  for player in textlst[0:10]:
    string = string  + str(i) + ". " + str(player[0]) + "\n"
    i+=1
  return string

In [ ]:
def display(club_name, position, age_min,age_max, val):
  if position == "Defenders":
    # print(type(club_name), type(position), type(age_min),type(age_max), type(val))
    lst = get_player_recs(df_22_std,club_name,["CB"],Def_essential_ft,def_weights_per_club, age_min, age_max, val)[0:70]
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  elif position == "Wingers":
    lst = get_player_recs(df_22_std,club_name,["RW","LW"],Wing_essential_ft,wing_weights_per_club, age_min, age_max, val)
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  elif position == "Strikers":
    lst = get_player_recs(df_22_std,club_name,["ST","CF","LF"],ST_essential_ft,striker_weights_per_club, age_min, age_max, val)
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  elif position == "Mid-Fielders":
    lst = get_player_recs(df_22_std,club_name,["LM","CM","RM", "CDM"],Mid_essential_ft,mid_weights_per_club, age_min, age_max, val)
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  elif position == "Centre Attacking Midfielder":
    lst = get_player_recs(df_22_std,club_name,["CAM"],Cam_essential_ft,cam_weights_per_club, age_min, age_max, val)
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  elif position == "Full Backs":
    lst = get_player_recs(df_22_std,club_name,["LB","RB","LWB","RWB"],Fullback_essential_ft,fullback_weights_per_club, age_min, age_max, val)
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  elif position == "Goalkeeper":
    lst = get_player_recs(df_22_std,club_name,["GK"],Gk_essential_ft,gk_weights_per_club, age_min, age_max, val)
    return cleanText(lst),{str(lst[0][0] + "'s Similarity"):lst[0][2][0],str(lst[0][0]+ "'s Strength"):lst[0][2][1]},{str(lst[10][0] + "'s Similarity"):lst[10][2][0],str(lst[10][0]+ "'s Strength"):lst[10][2][1]},{str(lst[20][0] + "'s Similarity"):lst[20][2][0],str(lst[20][0]+ "'s Strength"):lst[20][2][1]},{str(lst[30][0] + "'s Similarity"):lst[30][2][0],str(lst[30][0]+ "'s Strength"):lst[30][2][1]},{str(lst[40][0] + "'s Similarity"):lst[40][2][0],str(lst[40][0]+ "'s Strength"):lst[40][2][1]}
  return 0
# demo = gr.Interface(fn=display, inputs=[gr.Textbox(label="Enter Club Name"),
#                                         gr.Dropdown(
#                                           ["Defenders", "Wingers", "Strikers", "Mid_Fielders", "Center Attacking Midfielder", "Full Backs", "Goalkeeper"], label="Choose Player Position", 
#                                           info="Pick One",
#                                           # multiselect=True,
#                                         ),
#                                         gr.Slider(18, 100, label="Enter Minimum Age", info="Choose between 18 and 100"),
#                                         gr.Slider(18, 100,label="Enter Maximum Age", info="Choose between 18 and 100"),
#                                         gr.Slider(0, 500,label="Enter Value Willing to Pay(in millions)", info="Choose between 0 and 500")], 
#                     outputs=[gr.Textbox(label="Here are your Player Recommendations:"),
#                             # gr.Label(value={"cat": 0.732298102137154, "dog": 0.996334685537581, "fish": 0.1}),
#                             #  gr.Label(value={"cat": 0.7, "dog": 0.2, "fish": 0.1}),
#                             #  gr.Textbox(label="Here are your Player Recommendations:"),
#                              ],
#                     # gr.Label(value={"cat": 0.732298102137154, "dog": 0.996334685537581, "fish": 0.1}),
#                     allow_flagging="never",
#                     theme='freddyaboulton/dracula_revamped',
# )

demo = gr.Blocks(theme='freddyaboulton/dracula_revamped',
                 allow_flagging="never")

with demo:
    gr.Markdown(
        """
    # Scoutify!
    Fill the Form Below:
    """
    )
    with gr.Row():
      with gr.Column():
        club_input = gr.Textbox(label="Enter Club Name")
        pos_input = gr.Dropdown(["Defenders", "Wingers", "Strikers", "Mid_Fielders", "Center Attacking Midfielder", "Full Backs", "Goalkeeper"], label="Choose Player Position", 
                                info="Pick One",
                                # multiselect=True,
                              )
        minAge_input = gr.Slider(18, 100, label="Enter Minimum Age", info="Choose between 18 and 100")
        maxAge_input = gr.Slider(18, 100,label="Enter Maximum Age", info="Choose between 18 and 100")
        val_input = gr.Slider(0, 500,label="Enter Value Willing to Pay(in millions)", info="Choose between 0 and 500")
        text_button = gr.Button("Submit")
      with gr.Column():
        outLst = gr.Textbox(label="Here are your Player Recommendations:")
        card1 = gr.Label(label="Rank 1 Player")
        card2 = gr.Label(label="Rank 10 Player")
        card3 = gr.Label(label="Rank 20 Player")
        card4 = gr.Label(label="Rank 30 Player")
        card5 = gr.Label(label="Rank 40 Player")
    text_button.click(display, inputs=[club_input, pos_input, minAge_input, maxAge_input, val_input], outputs=[outLst,card1,card2,card3,card4,card5])

demo.launch(share=True) 
# demo.launch(share=False)